Remove files with generic file names

In [729]:
# %load_ext autoreload
# %reload_ext autoreload
# %autoreload 2

In [730]:
import os
os.chdir('C:\\Users\\WDG1DCR\\Desktop\\my_projects\\data_analysis\\data_cleanse')

import pandas as pd
from utility import generate
from prep import filterx

In [731]:
fileName = 'filelist_9_02_d'
df_og = pd.read_csv('datasets/' + fileName + '.csv', sep=';')

In [732]:
df = filterx.sld(df_og)

In [733]:
dups = df[df.duplicated(['name_low'], keep=False)]
dups = dups.sort_values(['name_low'])
dups = dups.reset_index(drop=True)

In [734]:
dups.shape

(1446, 7)

In [735]:
all_dups = dups['name_low'].value_counts(ascending=True).rename_axis('unique_values').reset_index(name='count')
all_dups = all_dups.iloc[::-1]
all_dups = all_dups.reset_index(drop=True)

In [736]:
all_dups.head()

,unique_values,count
0,kre fa 068005 - m6 spring washer.sldprt,7
1,test.sldprt,7
2,product.sldprt,7
3,9.sldprt,7
4,118974.sldprt,7


In [737]:
# create generic file name list, any file that does not contain at least one numerical digit in the file name
import re

number_fnames = []
generic_fnames = []

for i in all_dups['unique_values'].to_list():
    if re.search(r'\d', i.replace(' ', '')):
        number_fnames.append(i)
    else:
        generic_fnames.append(i)

In [738]:
# addend to the generic file name list and file name that is only a single digit (ie. 1.sldprt)
for i in all_dups['unique_values'].to_list():
    target = i.split('.')[0]
    if re.search(r'^\d$', target):
        print(i)
        generic_fnames.append(i)

9.sldprt
8.sldprt
1.sldasm


In [739]:
from collections import defaultdict

path_dict = {}
for i in generic_fnames:
    dups_trimmed = dups[dups['name_low'] == i]
    path_dict[i] = dups_trimmed['path'].to_list()

In [740]:
count = 0
for k, v in path_dict.items():
    print(k)
    count += len(v)
    for i in v:
        i = i.replace('\\\dcrsolidworks.patec.group\\Engineer1_E', 'R:')
        print(i)
    print()

test.sldprt
R:\Doc\SolidWorks\F37274 SUGAR FOODS MERGING CONVEYOR
R:\Doc\SolidWorks\P6155 MODEL A VARI-STRAIGHT
R:\Doc\SolidWorks\HS4836-4 TWO FLAP CLOSER (METRIC)
R:\Doc\SolidWorks\HS4873, HOME MARKET FOODS\4-HEAD ROTARY FEEDER
R:\Doc\SolidWorks\K22919 Hillshire Genesis Former
R:\Doc\SolidWorks\K22904 Bimbo Baking KFXWD OFC-FPC
R:\Doc\SolidWorks\K22256-CMD

product.sldprt
R:\Doc\SolidWorks\HS4972, RALCORP (SERVO HSR)
R:\Doc\SolidWorks\HS4974 ConAgra (Opp. hand Captain H)
R:\Doc\SolidWorks\HS4827
R:\Doc\SolidWorks\P6187 I-TRAK ENDLOAD CARTONER
R:\Doc\SolidWorks\HS4975 Nestle Gaffney Celox 9H\9in Celox feeder for Decatur 25 july 2008
R:\Doc\SolidWorks\Celox feeders\Study 3, Mid-range\9in Celox feeder
R:\Doc\SolidWorks\Hs4827-4 TWO FLAP CLOSER (IMPERIAL)

floor.sldprt
R:\Doc\SolidWorks\P6109 POLARIS II\110673 HVY DTY SPINDLE
R:\Doc\SolidWorks\P6106 RIGHT ANGLE LUGLESS\DESIGN REVIEW
R:\Doc\SolidWorks\Mainar\Infeed_Assy\Downloaded Components
R:\Doc\SolidWorks\POLARIS
R:\Doc\SolidWorks\Main

In [741]:
file_list = []

for k, v in path_dict.items():
    for i in v:
        path_and_file = i + '\\' + k
        file_list.append(path_and_file)

In [742]:
len(file_list)

520

In [1]:
###GENERATE BATCH FILE###
file_name = '9_03_files_to_remove_generic_names_script'
# generate.batch2(file_name, file_list)